In [1]:
%run heartbeat_slicer.ipynb

In [27]:
import os
import numpy as np
from matplotlib import pyplot as plt
import gudhi
import gudhi.representations.vector_methods as grvm
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_point_cloud

In [28]:
folder_path = "data_processed/mitdb"
record_names_path = "data_original/mitdb/RECORDS"

with open(record_names_path, 'r') as record_names_file:
    record_names = record_names_file.readlines()

record_names = [record_name[:-1] for record_name in record_names]

In [29]:
dataMitDB = []

for record_name in record_names:
    sliced = HeartbeatSlicer(folder_path + '/' + record_name + '_processed')
    dataMitDB.append(sliced.annotate_heartbeats())

In [30]:
#[patient][heartbeat_num] - [0]:heartbeat [1]:ann
dataMitDB[2][3][0].shape

(497,)

In [31]:
def make_point_cloud(heartbeat):

    embedding_dimension_periodic = 3
    embedding_time_delay_periodic = 8
    embedder_periodic = SingleTakensEmbedding(
        parameters_type="fixed",
        n_jobs=2,
        time_delay=embedding_time_delay_periodic,
        dimension=embedding_dimension_periodic,
    )
    point_cloud = embedder_periodic.fit_transform(heartbeat)
    plot_point_cloud(point_cloud)

    return point_cloud


In [32]:
def make_complex_w_pers_diag(point_cloud):
    point_cloud = point_cloud[None, :, :]

    homology_dimensions = [1]

    periodic_persistence = VietorisRipsPersistence(
        homology_dimensions=homology_dimensions, n_jobs=6
    )
    print("Persistence diagram for periodic signal")
    diag = periodic_persistence.fit_transform_plot(point_cloud)

    return diag


In [33]:
def make_betti_curve(heartbeat):
    # Convert the segment to a point cloud
    point_cloud = make_point_cloud(heartbeat)

    # Compute the Vietoris-Rips complex and its persistence diagram
    diag = make_complex_w_pers_diag(point_cloud)

    # Compute and return Betti curve

    bc = BettiCurve(n_jobs=-1)
    betti_curve = bc.fit_transform_plot(diag)
    return betti_curve

In [34]:
make_betti_curve(dataMitDB[2][2][0])

Persistence diagram for periodic signal


array([[[ 1,  8,  3,  1,  5,  8,  9,  7, 12,  9,  7,  6,  9,  7,  5,  7,
          7,  5,  4,  7,  4,  4,  4,  3,  3,  1,  0,  0,  0,  0,  0,  0,
          1,  2,  1,  2,  3,  3,  3,  3,  4,  2,  2,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  0]]])